<a href="https://www.kaggle.com/code/velimirkovacic/mn-0036533917-ferules?scriptVersionId=183359087" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Feature engineering, rules-based models and ensembles


In [ ]:
!pip install Wittgenstein

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from sklearn.svm import LinearSVC
import shap
import wittgenstein as lw
from IPython.display import display


First we will import the train dataset created in the first notebook. 

In [ ]:

df_train = pd.read_csv("../input/train-set-za-biljeznicu-2/train2.csv")
df_train['Date'] = pd.to_datetime(df_train['Date'])

df_test = pd.read_csv("../input/dapprojekt24-1/test.csv")



## 1. Cross-validation Function

The cross-validation function uses a five-fold cross-validation. The function takes a dataset (X y), splits it into 5 folds, tests 5 classifiers with default parameters and prints a table showing f1-scores. The calssifiers are: GaussianNB, LogisticRegression, RandomForestClassifier, ExtraTreesClassifier and XGBClassifier. 

In [ ]:
def test_model(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    return f1_score(y_test, y_pred)


def cross_val_5fold(X, y):
    models = [
    (GaussianNB(), "GaussianNB"),
    (LogisticRegression(), "LogisticRegression"),
    (RandomForestClassifier(), "RandomForestClassifier"),
    (ExtraTreesClassifier(), "ExtraTreesClassifier"),
    (XGBClassifier(), "XGBClassifier")
    ]

    data = {
            'Algorithm': [],
            'Fold 1': [],
            'Fold 2': [],
            'Fold 3': [],
            'Fold 4': [],
            'Fold 5': [],
            'Average': []
        }
    df = pd.DataFrame(data)


    for model, name in models: 
        print("Model:", name)
        five_folds = KFold(n_splits=5, shuffle=True)
        row = [name]
        i = 1
        for train, test in five_folds.split(X):
            X_train = X.iloc[train]
            X_test = X.iloc[test]
            y_train = y.iloc[train]
            y_test = y.iloc[test] 
            print("Fold", i)
            i+=1
            f1 = test_model(model, X_train, X_test, y_train, y_test)
            row += [f1]
        
        row += [np.average(row[1:])]
        df.loc[len(df)] = row
    print(df)



### Running the cross-validation function
We will use the five-fold cross-validation function on the default dataset.

In [ ]:
X = df_train.drop(columns=["Date", "Symbol", "Id", "Target"])
y = df_train["Target"]
cross_val_5fold(X, y)

### Submitting to Competition
The Random Forest Classifier has achieved the highest F1 score and we will thus submit it to competition as per the instructions. NaN values in the test set have been resolved using mean-fill.

In [ ]:
X_test = df_test.drop(columns=["Date", "Symbol", "Id"])
X_test = X_test.fillna(X_test.mean())
model = RandomForestClassifier()
model.fit(X, y)
y_test = model.predict(X_test)
submission = df_test.loc[:,df_test.columns.isin(('Id', ))]
submission.loc[:,'Target'] = y_test
submission.to_csv("submission_1.csv", index=None)
submission

## 2. New Features
### 2.1 Domain specific features
#### 2.1.1 Simple Moving Average SMA
SMA is calculated as the average closing price of the stock last N days. We will use N = 200 as it is the most commomn within trading circles. The 199 NaNs per stock will be filled with the Close values.

In [ ]:
df_train['SMA'] = df_train.groupby('Symbol')['Close'].rolling(window=200).mean().reset_index(0, drop=True)
df_train.loc[df_train['SMA'].isna(), 'SMA'] = df_train['Close']


#### 2.1.2 Exponential Moving Average (EMA)
Unlike SMA, EMA places greater importance on more recent values. It is a weighted average. We will use a 12 day span.

In [ ]:
df_train['EMA'] = df_train.groupby('Symbol')['Close'].apply(lambda x: x.ewm(span=12, adjust=False).mean()).reset_index(0, drop=True)

#### 2.1.3 Moving Average Convergence Divergence (MACD)
MACD is the difference between a 12 day span EMA and a 26 day span EMA. 

In [ ]:
df_train['MACD'] = df_train['EMA'] - df_train.groupby('Symbol')['Close'].apply(lambda x: x.ewm(span=26, adjust=False).mean()).reset_index(0, drop=True)

#### 2.1.4 Relative Strength Index (RSI)
RSI is a an indicator within range of 0 to 100 and is calculated as: 
$RSI = 100 - \frac{100}{1 + \frac{AvgGain}{AvgLoss}}.$ We will use a 14 day span.

The missing values will be filled with the mean value of that stock. We will do this for all other instances of this issue.



In [ ]:
change = df_train.groupby('Symbol')['Close'].diff()

df_train['Gain'] = change.apply(lambda x: x if x > 0 else 0)
df_train['Loss'] = change.apply(lambda x: -x if x < 0 else 0)

avgGain = df_train.groupby('Symbol')['Gain'].rolling(window=14, min_periods=1).mean().reset_index(drop=True)
avgLoss = df_train.groupby('Symbol')['Loss'].rolling(window=14, min_periods=1).mean().reset_index(drop=True)


df_train['RSI'] = 100 - (100 / (1 + avgGain/avgLoss))
df_train.loc[df_train['Loss'] < 1e-2, 'RSI'] = 100
df_train["RSI"] = df_train.groupby('Symbol')['RSI'].transform(lambda x: x.fillna(x.mean()))
df_train = df_train.drop(columns=['Gain', 'Loss'])

#### 2.1.5 On-Balance Volume (OBV)
OBV is calculated as:

$
OBV_i = OBV_{i -1} \begin{cases} 
Volume_i & \text{if } Close_i > Close_{i-1} \\
0 & \text{if } Close_i = Close_{i-1} \\
-Volume_i & \text{if } Close_i < Close_{i-1} 
\end{cases}
$

In [ ]:
df_train['Dir'] = change.apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
df_train['OBV'] = df_train.groupby('Symbol')['Volume'].transform(lambda x: x * df_train['Dir']).cumsum()

df_train = df_train.drop(columns=['Dir'])

In [ ]:
df_train

### 2.2 History related features


Here we will invent new history related features that may help in our predictions.

### 2.2.1 Close Entropy (CE)
We will calculate entropy of closing prices. We will use a 10 day span. Missing entropies will be filled by the mean of the specific stock.

In [ ]:
probs = df_train.groupby('Symbol')['Close'].transform(lambda x: x / x.sum())

df_train["CE"] = probs.rolling(window=10).apply(lambda x: -np.sum(x * np.log2(x))).reset_index(drop=True)
df_train["CE"] = df_train.groupby('Symbol')['CE'].transform(lambda x: x.fillna(x.mean()))

### 2.2.2 Average Open-Close (AOC)
This will be the average value of Open - Close. We will use a 20 day span.

In [ ]:
open_mean = df_train.groupby('Symbol')['Open'].rolling(window=20).mean().reset_index(drop=True)
close_mean = df_train.groupby('Symbol')['Close'].rolling(window=20).mean().reset_index(drop=True)

df_train["AOC"] = open_mean - close_mean
df_train["AOC"] = df_train.groupby('Symbol')['AOC'].transform(lambda x: x.fillna(x.mean()))

### 2.2.3 Close Variance (CV)
This will be the variance of Close. We will use a 30 day span.


In [ ]:
df_train["CV"] = df_train.groupby('Symbol')["Close"].rolling(window=30).var().reset_index(drop=True)
df_train["CV"] = df_train.groupby('Symbol')["CV"].transform(lambda x: x.fillna(x.mean()))

### 2.2.4 Mean Volume (MV)
This will be the mean of Volume. We will use a 15 day span.


In [ ]:
df_train["MV"] = df_train.groupby('Symbol')["Volume"].rolling(window=15).mean().reset_index(drop=True)
df_train["MV"] = df_train.groupby('Symbol')["MV"].transform(lambda x: x.fillna(x.mean()))

### 2.2.5 Volume Variance (VV)
This is the variance of Volume. We will use a 30 day span.

In [ ]:
df_train["VV"] = df_train.groupby('Symbol')["Volume"].rolling(window=30).var().reset_index(drop=True)
df_train["VV"] = df_train.groupby('Symbol')["VV"].transform(lambda x: x.fillna(x.mean()))

### 2.2.6 Average High-Low (AHL)
This will be the average value of High - Low. We will use a 20 day span.

In [ ]:
high_mean = df_train.groupby('Symbol')['High'].rolling(window=20).mean().reset_index(drop=True)
low_mean = df_train.groupby('Symbol')['Low'].rolling(window=20).mean().reset_index(drop=True)

df_train["AHL"] = high_mean - low_mean
df_train["AHL"] = df_train.groupby('Symbol')['AHL'].transform(lambda x: x.fillna(x.mean()))

### 2.2.7 High Variance (HV)
This is the variance of High. We will use a 20 day span.

In [ ]:
df_train["HV"] = df_train.groupby('Symbol')['High'].rolling(window=20).var().reset_index(drop=True)
df_train["HV"] = df_train.groupby('Symbol')['HV'].transform(lambda x: x.fillna(x.mean()))

### 2.2.8 Low Variance (LV)
This is the variance of Low. We will use a 20 day span.

In [ ]:
df_train["LV"] = df_train.groupby('Symbol')['Low'].rolling(window=20).var().reset_index(drop=True)
df_train["LV"] = df_train.groupby('Symbol')['LV'].transform(lambda x: x.fillna(x.mean()))

### 2.2.9 Volume Entropy (VE)
This is the entropy of Volume. We will use a 10 day span.

In [ ]:
probs = df_train.groupby('Symbol')['Volume'].transform(lambda x: x / x.sum())

df_train["VE"] = probs.rolling(window=10).apply(lambda x: -np.sum(x/np.sum(x) * np.log2(x/np.sum(x)))).reset_index(drop=True)
df_train["VE"] = df_train.groupby('Symbol')['VE'].transform(lambda x: x.fillna(x.mean()))

### 2.2.10 Open Entropy (OE)
This is the entropy of Open. We will use a 10 day span.

In [ ]:
probs = df_train.groupby('Symbol')['Open'].transform(lambda x: x / x.sum())

df_train["OE"] = probs.rolling(window=10).apply(lambda x: -np.sum(x/np.sum(x) * np.log2(x/np.sum(x)))).reset_index(drop=True)
df_train["OE"] = df_train.groupby('Symbol')['OE'].transform(lambda x: x.fillna(x.mean()))

In [ ]:
df_train

### Normalization

We will create a new dataset with the 15 new features and then normalize them with Min-Max normalization.

In [ ]:
df = df_train.drop(columns=["Open", "High", "Low", "Close", "Adj Close", "Id", "Date", "Volume"])

for i in df.columns:
    if i == 'Symbol':
        continue
    df[i] = df.groupby('Symbol')[i].transform(lambda x: ((x - x.min() )/ (x.max() - x.min()))).reset_index(drop=True)

df = df.drop(columns=['Symbol'])

In [ ]:
df

### Running the Cross-Validation Function

We will find which model yields the highest F1 Score.

In [ ]:
X = df.drop(columns=["Target"])
y = df["Target"]
cross_val_5fold(X, y)

Extra Trees Classifier has yielded the best F1 score. We will train the model and send it to competition.

### Submitting to Competition
We will have to create all the features for the test set and then normalize them.

In [ ]:
df_test['SMA'] = df_test.groupby('Symbol')['Close'].rolling(window=200).mean().reset_index(0, drop=True)
df_test.loc[df_test['SMA'].isna(), 'SMA'] = df_test['Close']
df_test['EMA'] = df_test.groupby('Symbol')['Close'].apply(lambda x: x.ewm(span=12, adjust=False).mean()).reset_index(0, drop=True)
df_test['MACD'] = df_test['EMA'] - df_test.groupby('Symbol')['Close'].apply(lambda x: x.ewm(span=26, adjust=False).mean()).reset_index(0, drop=True)
change = df_test.groupby('Symbol')['Close'].diff()
df_test['Gain'] = change.apply(lambda x: x if x > 0 else 0)
df_test['Loss'] = change.apply(lambda x: -x if x < 0 else 0)
avgGain = df_test.groupby('Symbol')['Gain'].rolling(window=14, min_periods=1).mean().reset_index(drop=True)
avgLoss = df_test.groupby('Symbol')['Loss'].rolling(window=14, min_periods=1).mean().reset_index(drop=True)
df_test['RSI'] = 100 - (100 / (1 + avgGain/avgLoss))
df_test.loc[df_test['Loss'] < 1e-2, 'RSI'] = 100
df_test["RSI"] = df_test.groupby('Symbol')['RSI'].transform(lambda x: x.fillna(x.mean()))
df_test = df_test.drop(columns=['Gain', 'Loss'])
df_test['Dir'] = change.apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
df_test['OBV'] = df_test.groupby('Symbol')['Volume'].transform(lambda x: x * df_test['Dir']).cumsum()
df_test = df_test.drop(columns=['Dir'])
probs = df_test.groupby('Symbol')['Close'].transform(lambda x: x / x.sum())
df_test["CE"] = probs.rolling(window=10).apply(lambda x: -np.sum(x * np.log2(x))).reset_index(drop=True)
df_test["CE"] = df_test.groupby('Symbol')['CE'].transform(lambda x: x.fillna(x.mean()))
open_mean = df_test.groupby('Symbol')['Open'].rolling(window=20).mean().reset_index(drop=True)
close_mean = df_test.groupby('Symbol')['Close'].rolling(window=20).mean().reset_index(drop=True)
df_test["AOC"] = open_mean - close_mean
df_test["AOC"] = df_test.groupby('Symbol')['AOC'].transform(lambda x: x.fillna(x.mean()))
df_test["CV"] = df_test.groupby('Symbol')["Close"].rolling(window=30).var().reset_index(drop=True)
df_test["CV"] = df_test.groupby('Symbol')["CV"].transform(lambda x: x.fillna(x.mean()))
df_test["MV"] = df_test.groupby('Symbol')["Volume"].rolling(window=15).mean().reset_index(drop=True)
df_test["MV"] = df_test.groupby('Symbol')["MV"].transform(lambda x: x.fillna(x.mean()))
df_test["VV"] = df_test.groupby('Symbol')["Volume"].rolling(window=30).var().reset_index(drop=True)
df_test["VV"] = df_test.groupby('Symbol')["VV"].transform(lambda x: x.fillna(x.mean()))
high_mean = df_test.groupby('Symbol')['High'].rolling(window=20).mean().reset_index(drop=True)
low_mean = df_test.groupby('Symbol')['Low'].rolling(window=20).mean().reset_index(drop=True)
df_test["AHL"] = high_mean - low_mean
df_test["AHL"] = df_test.groupby('Symbol')['AHL'].transform(lambda x: x.fillna(x.mean()))
df_test["HV"] = df_test.groupby('Symbol')['High'].rolling(window=20).var().reset_index(drop=True)
df_test["HV"] = df_test.groupby('Symbol')['HV'].transform(lambda x: x.fillna(x.mean()))
df_test["LV"] = df_test.groupby('Symbol')['Low'].rolling(window=20).var().reset_index(drop=True)
df_test["LV"] = df_test.groupby('Symbol')['LV'].transform(lambda x: x.fillna(x.mean()))
probs = df_test.groupby('Symbol')['Volume'].transform(lambda x: x / x.sum())
df_test["VE"] = probs.rolling(window=10).apply(lambda x: -np.sum(x/np.sum(x) * np.log2(x/np.sum(x)))).reset_index(drop=True)
df_test["VE"] = df_test.groupby('Symbol')['VE'].transform(lambda x: x.fillna(x.mean()))
probs = df_test.groupby('Symbol')['Open'].transform(lambda x: x / x.sum())
df_test["OE"] = probs.rolling(window=10).apply(lambda x: -np.sum(x/np.sum(x) * np.log2(x/np.sum(x)))).reset_index(drop=True)
df_test["OE"] = df_test.groupby('Symbol')['OE'].transform(lambda x: x.fillna(x.mean()))


In [ ]:
df_test = df_test.drop(columns=["Open", "High", "Low", "Close", "Adj Close", "Date", "Volume"])

for i in df_test.columns:
    if i == 'Symbol' or i == 'Id':
        continue
    df_test[i] = df_test.groupby('Symbol')[i].transform(lambda x: ((x - x.min() )/ (x.max() - x.min()))).reset_index(drop=True)

df_test = df_test.drop(columns=['Symbol'])

In [ ]:
X_test = df_test.drop(columns=['Id'])
X_test = X_test.fillna(X_test.mean())
model = ExtraTreesClassifier()
model.fit(X, y)
y_test = model.predict(X_test)
submission = df_test.loc[:,df_test.columns.isin(('Id', ))]
submission.loc[:,'Target'] = y_test
submission.to_csv("submission_2.csv", index=None)
submission

## 3. Wrapper Method
By training a Linear SVC we will see which features contribute more and which feature contribute less to the F1 score.
We will train using 14 features, remove the least contributing one, then 13 features unitl and so on we are left with only 12 features.

In [ ]:
def wrapper(X, y):
    model = LinearSVC()
    print("Column", "F1 Score")
    for column in X.columns:
        X_wrapper = X.drop(columns=[column])
        model.fit(X_wrapper, y)
        y_pred = model.predict(X_wrapper)
        f1 = f1_score(y, y_pred)
        print(column, f1)

In [ ]:
X = df.drop(columns=["Target"])
y = df["Target"]
wrapper(X, y)

The feature without which the score is highest is Volume Entropy (VE), we will remove it.

In [ ]:
X = X.drop(columns=["VE"])
wrapper(X, y)

Removing LV, HV, MV and CV yields the same F1 score. We will remove LV.

In [ ]:
X = X.drop(columns=["LV"])
wrapper(X, y)

MV, HV and CV have the same F1 score, we will now remove MV.

In [ ]:
X = X.drop(columns=["MV"])
X

We are now left with 12 features.

### Running the Cross-Validation Function

In [ ]:
cross_val_5fold(X, y)

ExtraTreesClassifier has once again proved to be the best for this.

### Submitting to Competition

In [ ]:
X_test = df_test.drop(columns=['VE', 'LV', 'MV', 'Id'])
X_test = X_test.fillna(X_test.mean())
model = ExtraTreesClassifier()
model.fit(X, y)
y_test = model.predict(X_test)
submission = df_test.loc[:,df_test.columns.isin(('Id', ))]
submission.loc[:,'Target'] = y_test
submission.to_csv("submission.csv", index=None)
submission

## 4. Explainability with SHAP

In [ ]:
explainer = shap.TreeExplainer(model)
explainer

We will chose 10 rows of which the last 5 is all with Target equal to 1 and the first 5 with at least one with Target equal to 0 

In [ ]:
selected = X.iloc[161312:161322]
print(df_train.iloc[161312:161322]["Target"])
selected

We will extract the shap values and create a force_plot from these 10 samples.

In [ ]:
shap_values = []
for i in range(len(selected)):
    shap_values += [explainer.shap_values(selected.iloc[i])]


### Force Plots

In [ ]:
shap.initjs()
for value in shap_values:
    display(shap.force_plot(explainer.expected_value[1], value[0]))

For samples with Target = 0 the most impactful feature is CV (Close Variance), followed by OBV (On-Balance Volume).
For samples with Target = 1 the most impactful feature is EMA (Exponential Moving Average), followed by MACD (Moving average Convergence/Divergence) and AHL (Average High - Low).

In [ ]:
def visualize(series):
    fig, ax = plt.subplots(1, 1, figsize=(10, 5))

    for col in ["CV", "OBV", "MACD", "EMA", "AHL"]:
        ax.plot(range(len(series)), series[col], label=col)
    ax.set_title("10 day visualization")
    ax.set_xlabel("Day")
    ax.set_ylabel("Feature")
    ax.legend()

    plt.tight_layout()
    plt.show()
    
visualize(selected)

## 5. RIPPER Optimization

We will choose NVIDIA stock.

In [ ]:
df_test

In [ ]:
df = df_train.drop(columns=["Id", "Open", "High", "Low", "Close", "Adj Close", "Date", "Volume", "VE", "LV", "MV"])
for i in df.columns:
    if i == 'Symbol' or i == 'Id':
        continue
    df[i] = df_train.groupby('Symbol')[i].transform(lambda x: ((x - x.min() )/ (x.max() - x.min()))).reset_index(drop=True)

X = df[df_train["Symbol"] == "NVDA"]
X

In [ ]:
y = X["Target"]
X = X.drop(columns=["Target", "Symbol"])
X

We do a 67%-33% train-test split.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
model = lw.RIPPER()
model.fit(X_train, y_train)

We do a prediction on the test set.

In [ ]:
y_pred = model.predict(X_test)
f1 = f1_score(y_test, y_pred)
print("F1 Score:", f1)

We also print the rules of the RIPPER model.

In [ ]:
for rule in model.ruleset_:
    print(rule)

The rules seem to be abundant and convoluted. However it is much more interpretable than those of other models.

For example let's look at rule [OBV=0.24-0.35^EMA=0.012-0.019], it is applied when OBV is in range $[0.24, 0.35]$ and EMA is in range $[0.012, 0.019]$.

### Grid search optimization of hyperparameters
We will use grid search to find the optimal values of parameters k, prune_size and dl_allowance

In [ ]:
def grid_search(max_rules=None, max_rule_conds=None):
    ks = [1, 2, 3, 4, 5]
    prune_sizes = [0.1, 0.15, 0.2, 0.25, 0.3]
    dl_allowances = [16, 32, 64, 100, 128, 180, 256]

    best_f1 = 0
    best_params = None

    print("K", "prune_size", "dl_allowance", "F1 Score")
    for k in ks:
        for prune_size in prune_sizes:
            for dl_allowance in dl_allowances:
                model = lw.RIPPER(max_rules=max_rules, max_rule_conds=max_rule_conds, k=k, prune_size=prune_size, dl_allowance=dl_allowance)
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)
                f1 = f1_score(y_test, y_pred)

                print(k, prune_size, dl_allowance, f1)

                if f1 > best_f1:
                    best_f1 = f1
                    best_params = {'k': k, 'prune_size': prune_size, 'dl_allowance': dl_allowance}
    return best_f1, best_params

In [ ]:
best_f1, best_params = grid_search()

### Best Model Parameters


In [ ]:
print(best_f1)
best_params

### Best Model Rules

In [ ]:
model = lw.RIPPER(k=best_params["k"], prune_size=best_params["prune_size"], dl_allowance=best_params["dl_allowance"])
model.fit(X_train, y_train)

In [ ]:
for rule in model.ruleset_:
    print(rule)

### Limited RIPPER
We will see what happens when we limit the number of rules to 3 and rule conditions to 2 and see what happens.


In [ ]:
model = lw.RIPPER(max_rules=3, max_rule_conds=2, k=best_params["k"], prune_size=best_params["prune_size"], dl_allowance=best_params["dl_allowance"])
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
f1 = f1_score(y_test, y_pred)
print(f1)
for rule in model.ruleset_:
    print(rule)

The best model's accuracy drops dramatically. We will repeat the grid search.

In [ ]:
best_f1, best_params = grid_search(max_rules=3, max_rule_conds=2)

The grid search confirms our suspicion that there is not much that can be done with 3 rules and 2 conditions per rule.